In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import truncnorm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.model_selection import GridSearchCV, PredefinedSplit

from pysim import SimBoostRegressor, SimBoostClassifier

## Regression

```python
from pysim import SimBoostRegressor
clf = SimBoostRegressor(n_estimators, val_ratio=0.2, degree=2, knot_num=20, reg_lambda=0.1, reg_gamma=10, random_state=0)
```

In [2]:
def data_generator1_reg(random_state=0):
    
    np.random.seed(random_state)
    # data generation
    beta1 = np.array([.2, 0.3, 0.5, 0, 0, 0, 0, 0, 0, 0])
    beta2 = np.array([0, .2, 0.3, 0.5, 0, 0, 0, 0, 0, 0])
    beta3 = np.array([0, 0, 0.2, 0.3, 0.5, 0, 0, 0, 0, 0])
    
    beta = np.vstack([beta1,beta2,beta3])
    model_list = [lambda x: 0.2*np.exp(-4*x), lambda x: 3*x**2, lambda x: 2.5*np.sin(1.5 * np.pi*x)]
    
    x = truncnorm.rvs(a=-3,b=3,loc = 0, scale=1/3, size=(20000,10),random_state=random_state)
    noise = np.random.randn(20000).reshape(-1, 1)
    y = np.reshape(0.2 * np.exp(-4 * np.dot(x, beta1)) + \
                   3 * (np.dot(x, beta2))**2 + 2.5 * np.sin(np.pi * 1.5 * np.dot(x, beta3)), [-1, 1]) + noise
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=random_state)
    return beta, model_list, train_x, test_x, train_y, test_y

In [5]:
clf._validate_input(train_x, train_y)

(array([[ 0.37631248,  0.0880867 , -0.13741911, ..., -0.66403317,
          0.07519673, -0.43819955],
        [-0.57459581, -0.7757345 , -0.33681965, ..., -0.42656552,
         -0.16748566, -0.08869435],
        [ 0.05396652, -0.01398677, -0.33126588, ...,  0.61227938,
          0.23554845,  0.22353064],
        ...,
        [ 0.06140863, -0.07914898,  0.75493424, ...,  0.00444913,
         -0.06320347, -0.22817666],
        [ 0.38711759, -0.19909707, -0.66455354, ..., -0.43265208,
          0.01536014, -0.12419947],
        [ 0.11023205,  0.11547656, -0.25433631, ..., -0.21079951,
          0.72843839, -0.50406408]]), array([[-0.08358343],
        [ 0.7123498 ],
        [ 1.84646835],
        ...,
        [-3.03529853],
        [-0.64836179],
        [-2.58591141]]))

In [4]:
beta, model_list, train_x, test_x, train_y, test_y = data_generator1_reg()

clf = SimBoostRegressor(n_estimators=5, knot_num=20, reg_lambda=0.05, reg_gamma=10)

clf.fit(train_x, train_y)
clf.visualize()
mean_squared_error(test_y, clf.predict(test_x))

ValueError: operands could not be broadcast together with shapes (12800,10) (204800000,1) 

In [ ]:
clf.visualize_val_perf()

## LogitBoost

```python
from pysim import SimBoostClassifier
clf = SimBoostClassifier(n_estimators, val_ratio=0.2, degree=2, knot_num=20, reg_lambda=0.1, reg_gamma=10, random_state=0)
```

In [ ]:
def data_generator1_cls(random_state=0):
    
    np.random.seed(random_state)
    # data generation
    beta1 = np.array([.2, 0.3, 0.5, 0, 0, 0, 0, 0, 0, 0])
    beta2 = np.array([0, .2, 0.3, 0.5, 0, 0, 0, 0, 0, 0])
    beta3 = np.array([0, 0, 0.2, 0.3, 0.5, 0, 0, 0, 0, 0])
    
    beta = np.vstack([beta1,beta2,beta3])
    model_list = [lambda x: 0.2*np.exp(-4*x), lambda x: 3*x**2, lambda x: 2.5*np.sin(1.5 * np.pi*x)]
    
    x = truncnorm.rvs(a=-3, b=3, loc = 0, scale=1/3, size=(10000, 10),random_state=random_state)
    noise = np.random.randn(10000).reshape(-1, 1)
    y = np.reshape(0.2 * np.exp(-4 * np.dot(x, beta1)) + \
                   3 * (np.dot(x, beta2))**2 + 2.5 * np.sin(np.pi * 1.5 * np.dot(x, beta3)), [-1, 1]) + noise
    y[y <= 0] = 0
    y[y > 0] = 1
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=random_state)
    return beta, model_list, train_x, test_x, train_y, test_y

In [ ]:
beta, model_list, train_x, test_x, train_y, test_y = data_generator1_cls()

clf = SimBoostClassifier(n_estimators=5, knot_num=20, reg_lambda=0.05, reg_gamma=10)
clf.fit(train_x, train_y)
clf.visualize()
roc_auc_score(test_y, clf.predict_proba(test_x))